**IMPORTACIÓN DE LIBRERIAS**

In [24]:
import numpy as np
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout #RN convulocional
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**CONFIGURAR RUTA DEL DATASET**

In [25]:
# Ruta del dataset descargando desde la sección 'add dataset' en kaggle notebook
dataset_path ="/kaggle/input/brain-tumor-mri-dataset"

# Verificamos las carpetas del dataset 
os.listdir(dataset_path)

['Training', 'Testing']

**PARAMETROS DE PREPROCESAMIENTO**

In [26]:
# Dimensiones de las imagenes (todos seran redimensionados)
img_width = 150
img_height = 150

# Definir rutas especificas
train_dir = os.path.join(dataset_path, 'Training')
test_dir = os.path.join(dataset_path, 'Testing')


**AUMENTACIÓN DE DATOS**

In [27]:
# Generador de imagenes de entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,       #Normalizar
    rotation_range=20,   #Rotar Ligeramente
    zoom_range=0.15,     #Zoom Aleatorio
    width_shift_range=0.2,  #Desplazamiento Horizontal
    height_shift_range=0.2, #Desplazamiento Vertical
    shear_range=0.15,    #Cizalladura
    horizontal_flip=True,   #Volteo Horizontal
    fill_mode="nearest"     #Relleno de pixeles vacios
)

# Generador de imagenes de prueba (solo normalizar)
test_datagen = ImageDataGenerator(rescale=1/255)

# Crear datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

Found 5712 images belonging to 4 classes.
Found 5712 images belonging to 4 classes.


**CREAR EL MODELO CNN**

In [28]:
model = Sequential ([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') #Salida: Tumor o no Tumor solo 2 estados
])

**COMPILAR MODELO**

In [29]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

**ENTRENAMIENTO DEL MODELO**

In [ ]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=test_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 217s 1s/step - accuracy: 0.2211 - loss: -137694704.0000 - val_accuracy: 0.2344 - val_loss: -8730300416.0000
Epoch 2/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 244s 1s/step - accuracy: 0.2379 - loss: -50360229888.0000 - val_accuracy: 0.2344 - val_loss: -582918668288.0000
Epoch 3/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 206s 1s/step - accuracy: 0.2309 - loss: -1141583773696.0000 - val_accuracy: 0.2344 - val_loss: -5549402882048.0000
Epoch 4/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.2323 - loss: -8201747038208.0000 - val_accuracy: 0.2344 - val_loss: -25131296489472.0000
Epoch 5/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 236s 1s/step - accuracy: 0.2397 - loss: -30605433634816.0000 - val_accuracy: 0.2344 - val_loss: -77368039309312.0000
Epoch 6/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step - accuracy: 0.2291 - loss: -88815024734208.0000 - val_accuracy: 0.2344 - val_loss: -189273621921792.0000
Epoch 7/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - accuracy: 0.2412 - 

**GRAFICAR PRECISIÓN Y PERDIDA**

In [ ]:
# Precisión
plt.plot(history.history['accuracy'], label='Precisión Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión Validación')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.grid(True)
plt.show()

# Pérdida
plt.plot(history.history['loss'], label='Pérdida Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida Validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.grid(True)
plt.show()


**PREDICCIÓN DE UNA SOLA IMAGEN**

In [ ]:
# Ruta de una imagen de prueba 
test_image_path = os.path.join(test_dir, 'notumor', 'Te-no_0010.jpg') # Puedes cambiar la subcarpeta

# Cargar Imagen
img = cv2.imread(test_image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #Convertir BGR a RGB
img_resized = cv2.resize(img, (img_width, img_height))

# Normalizar y expandir dimensiones
img_normalized = img_resized / 255.0
img_input = np.expand_dims(img_normalized, axis=0) # (1, 150, 150,3)

# Realizar predicción
prediction = model.predict(img_input)[0][0]

# Interpretar Predicción
if prediction > 0.5:
    resultado = "Tumor detectado"
    color = "red"
else:
    resultado = "No se detecto tumor"
    color = "green"

# Mostrar Imagen y Resultado
plt.figure(figsize=(5,5))
plt.imshow(img.resized)
plt.title(f"{resultado} ({prediction:.2f})", color=color)
plt.axis('off')
plt.show()